In [ ]:
import sqlite3
import random
import itertools
import subprocess
import os
import shlex
import time
import pickle
import joblib
from multiprocessing import Pool

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [ ]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
from imageio import imread

In [ ]:
from skimage.transform import resize
from sklearn.decomposition import PCA, IncrementalPCA

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
# import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)

In [ ]:
# path = "/home/rte/data/images/web/120k/" + "2971229.jpg"
dim = 256

In [ ]:
filepaths = []

with open("/home/rte/data/paths/all_converted_jpg_images_shuf.txt", "r") as f:
    lines = f.readlines()
    print("length:",len(lines))
    print(lines[0])
for l in lines:
    # substrings = l.rsplit(",", 1)
    filepaths.append(l.strip())
    # image_ids.append(substrings[1].strip())

In [ ]:
# image_folder = "/mnt/hd2/images/all/"
image_folder = "/home/rte/data/images/web/120k/"

In [ ]:
# get the image paths for all images that we have features for
images = []

for i, f in enumerate(filepaths[:]):
    images.append(image_folder + f)
print(len(images))
print(images[:10])

In [ ]:
images[0]

In [ ]:
# original, crop, squish, crop_resized, pad
# crop resizes to same aspect ratio that fits first then takes crop from centre
columns = 4
rows = 10
# fig = plt.figure(figsize=(8,8))
fig, ax = plt.subplots(rows, columns, figsize=(8,8))
offset = 90

# fig.suptitle(title_string, y=1.0)
ax[0, 0].set_title("original", pad=20)
ax[0, 1].set_title("crop", pad=20)
ax[0, 2].set_title("squish", pad=20)
# ax[0, 3].set_title("crop_resize", pad=20)
ax[0, 3].set_title("pad", pad=20)

for y in range(rows):
    for x in range(columns):
        ax[y, x].axis('off')
        
# for x in range(columns):
#     ax[0, x].set_title("path:" + path.rsplit("/")[0], pad=20)

# def load_img(path, dim):
for i, path in enumerate(images[offset:offset+rows]):
    im = Image.open(path).convert('RGB')
    img_x, img_y = im.size
    print(im.size)
    ax[i, 0].imshow(im, cmap='Greys_r')
#     ax[0, 0].set_title("query image ID: " + path.rsplit(".")[0], pad=20)

    # im.show()
    x_bigger = True if img_x > img_y else False
    print("x_bigger:",x_bigger)

    if True:
        ratio = img_x / img_y

        if x_bigger:
            factor = float(dim) / img_y 
        else:
            factor = float(dim) / img_x
        new_x = int(img_x * factor)
        new_y = int(img_y * factor)

        resized = im.resize((new_x, new_y), Image.ANTIALIAS)
        # resized.show()

        left = (new_x - dim)/2
        top = (new_y - dim)/2
        right = (new_x + dim)/2
        bottom = (new_y + dim)/2

        cropped = resized.crop((left, top, right, bottom))
#         cropped.show()
        ax[i, 1].imshow(cropped, cmap='Greys_r')


        print("factor", factor)
        print("ratio", ratio)
        print(x_bigger)
        print("img_x", img_x)
        print("img_y", img_y)
        print(new_x, new_y)
        print("cropped", cropped.size)

    # just resize to fill the box
    just_resize = im.resize((dim, dim), Image.ANTIALIAS)
    #     just_resize.show()
    ax[i, 2].imshow(just_resize, cmap='Greys_r')


    if True:
        if x_bigger:
            factor = float(dim) / img_x
        else:
            factor = float(dim) / img_y
        new_x = int(img_x * factor)
        new_y = int(img_y * factor)
        print("factor", factor)
        print("img_x", img_x)
        print("img_y", img_y)
        print("new_x", new_x)
        print("new_y", new_y)
        crop_resized = im.resize((new_x, new_y), Image.ANTIALIAS)
#         ax[i, 3].imshow(crop_resized, cmap='Greys_r')


        # pad
        padded = Image.new("RGB", (dim, dim))
        padded.paste(crop_resized, ((dim - new_x)//2, (dim - new_y)//2))
#         padded.show()
        ax[i, 3].imshow(padded, cmap='Greys_r')


    # return just_resize
    
plt.tight_layout(pad=0.5)
savename = f"crop_comparison_o{offset}-n{rows+offset}.jpg"
plt.savefig(savename, dpi=150, bbox_inches='tight')

In [ ]:
# original, crop, crop_resized, squish, pad
# crop resizes to same aspect ratio that fits first then takes crop from centre
w = 4
h = 4